<a href="https://colab.research.google.com/github/haleelsada/NLP-some_tools/blob/main/NLP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NLP-Challenge**

Challenge: 
3 groups of text are given.

From text group 1,
1.	identify entities (person, place, thing, location, work of art)
2.	Categorize them into ‘living entity’, ‘person’ or ‘object’. 


In [ ]:
text1='''The Office is an American mockumentary sitcom television series that depicts the everyday work lives of office employees in the Scranton, Pennsylvania, branch of the 
fictional Dunder Mifflin Paper Company. It aired on NBC from March 24, 2005, to May 16, 2013, lasting a total of nine seasons.[1] Based on the 2001–2003 BBC series of the same 
name, it was adapted for American television by Greg Daniels, a veteran writer for Saturday Night Live, King of the Hill, and The Simpsons. It was co-produced by Daniels's 
Deedle-Dee Productions, and Reveille Productions (later Shine America), in association with Universal Television. The original executive producers were Daniels, Howard Klein, 
Ben Silverman, Ricky Gervais, and Stephen Merchant, with numerous others being promoted in later seasons. 
P. Padmarajan (23 May 1945 – 23 January 1991) was an Indian film maker, screenwriter and author who was known for his landmark work in Malayalam literature and Malayalam cinema. 
Padmarajan was the founder of a new school of film making in Malayalam cinema, along with Bharathan and K. G. George, in the 1980s, which created groundbreaking films that were 
widely received while also being critically acclaimed. Known for his classic works, he is widely regarded as one of the greatest film makers of Indian Cinema of all time. He is 
known for the mastery in screenwriting in Malayalam Cinema. Padmarajan was noted for his fine and detailed screenwriting and expressive direction style. Padmarajan made some of 
the landmark motion pictures in Malayalam cinema, including critically acclaimed works like Moonnam Pakkam (1988), Innale (1989), Season (1989) and Njan Gandharvan (1991). He 
wrote several short stories which were unique in content and presentation. His novels handled the darkest emotions and are considered as classics.'''

* Identify entities

In [ ]:
#import spacy and 'en_core_web_sm' ,dissable all unwanted pipeline functions for good speed
import spacy
nlp=spacy.load('en_core_web_sm')
nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])
import random
from spacy.util import minibatch, compounding
from pathlib import Path
textt=nlp(text1)

In [ ]:
print(len(textt.ents))
for i in textt.ents:
  print(i,i.label_)

49
American NORP
Scranton GPE
Pennsylvania GPE
Dunder Mifflin Paper Company ORG
NBC ORG
March 24, 2005 DATE
May 16, 2013 DATE
nine CARDINAL
BBC ORG
American NORP
Greg Daniels PERSON
Saturday Night Live, King of the Hill WORK_OF_ART
Simpsons PERSON
Daniels PERSON
Deedle-Dee Productions ORG
Reveille Productions ORG
Shine America LOC
Universal Television ORG
Daniels PRODUCT
Howard Klein PERSON
Ben Silverman PERSON
Ricky Gervais PERSON
Stephen Merchant PERSON
later seasons DATE
P. Padmarajan PERSON
23 CARDINAL
May 1945 DATE
23 CARDINAL
January 1991 DATE
Indian NORP
Malayalam ORG
Malayalam NORP
Padmarajan PERSON
Malayalam ORG
Bharathan PERSON
K. G. George PERSON
the 1980s DATE
Indian NORP
Malayalam Cinema PERSON
Padmarajan PERSON
Padmarajan PERSON
Malayalam ORG
Moonnam Pakkam PERSON
1988 DATE
Innale ORG
1989 DATE
1989 DATE
Njan Gandharvan ORG
1991 DATE


As we can see this model is not good, since it can't identify The office as an entity and it find The simpsons as person and few more mistakes,
so train it on new data for better named entity recognition

In [ ]:
DATA = [
              ("Malayalam is a language spoken by south indians", {"entities": [(0, 9, "LANGUAGE")]}),
              ("A lot of people speaks Malayalam", {"entities": [(24, 33, "LANGUAGE")]}),
              ("I love Malayalam movies", {"entities": [(8,17, "LANGUAGE")]}),
              ("Tamil was derived from sanskrit", {"entities": [(0,5, "LANGUAGE")]}),
              ("He is a Malayalam actor", {"entities": [(9,18, "LANGUAGE")]}),
              ("Manichithrathazh (1997) is an amazing movie", {"entities": [(0,17, "WORK_OF_ART")]}),
              ("Inception (2010) was an exceptional movie", {"entities": [(0,9, "WORK_OF_ART")]}),
              ("Malayalam Cinema industry is very famous for its art works", {"entities": [(0,16, "ORG")]}),
              ("Johns is a friend of me", {"entities": [(0,5, "PERSON")]}),
              ("I am Davy jones ", {"entities": [(5,15, "PRODUCT")]}),
              ("Thaniyavarthanam (1999) and vatsalayam (1991) are exceptional work of mamooty", {"entities": [(0,16, "WORK_OF_ART")]}),
              ("Thaniyavarthanam (1999) and vatsalayam (1991) are exceptional work of mamooty", {"entities": [(28,37, "WORK_OF_ART")]}),
              ("he was a writer for The Batman, The Priest", {"entities": [(20,29, "WORK_OF_ART")]}),
              ("Maheshinte Prathikaram (2017) is a wonderfull movie experience", {"entities": [(0,21, "WORK_OF_ART")]}),
              ("I watched Njan Prakashan (2018) ", {"entities": [(10,24, "WORK_OF_ART")]})
              ]

In [ ]:
# Training on DATA
# Getting the pipeline component
# Adding labels to the `ner`
ner=nlp.get_pipe("ner")
for _, annotations in DATA:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 30 iterations
  for iteration in range(30):

    # shuufling examples  before every iteration
    random.shuffle(DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )


In [ ]:
# Again find entities after training
texttt=nlp(text1)
dictionry={}
print(len(texttt.ents))
for i in texttt.ents:
  print(i,i.label_)

36
Office PERSON
American NORP
Scranton GPE
Pennsylvania GPE
Dunder Mifflin Paper Company ORG
NBC ORG
March 24, 2005 DATE
May 16, 2013 DATE
American NORP
Greg Daniels PERSON
Saturday Night Live, King of the Hill WORK_OF_ART
Simpsons PERSON
Daniels PERSON
Deedle-Dee Productions ORG
Reveille Productions ORG
Shine America ORG
Universal Television ORG
Daniels PERSON
Howard Klein PERSON
Ben Silverman PERSON
Ricky Gervais PERSON
Stephen Merchant PERSON
Indian NORP
Malayalam LANGUAGE
Malayalam NORP
Padmarajan PERSON
Malayalam NORP
Bharathan PERSON
Indian Cinema ORG
Malayalam Cinema ORG
Padmarajan PERSON
Padmarajan PERSON
Malayalam LANGUAGE
Moonnam Pakkam WORK_OF_ART
Innale WORK_OF_ART
Njan Gandharvan WORK_OF_ART


It is better than the first model ,it has some limitations too as it is not showing any dates and all

In [ ]:
# Catogerize and store them into topics we want in a dictionry
# There is not correct defintion for thing so not adding organisation and language to it
dict={'person':[],'place':[],'thing':[],'location':[],'work of art':[]}
for i in texttt.ents:
  if i.label_=='PERSON':
    dict['person'].append(i)
  elif i.label_=='GPE':
    dict['place'].append(i)
    dict['location'].append(i)
  elif i.label_=='PRODUCT':
    dict['thing'].append(i)
  elif i.label_=='WORK_OF_ART':
   dict['work of art'].append(i)

In [ ]:
# Categorize first answer to person, living entity ,object
dict2={'living entity':[], 'person':[],'object':[]}
dict2['person']=dict['person']
dict2['object']=dict['work of art']+dict['thing']+dict['location']


In [ ]:
print('text-1\nidentify entities (person, place, thing, location, work of art)')
for i,j in dict.items():
  print(i,':',j)
print('Categorize them into ‘living entity’, ‘person’ or ‘object’.')
for i,j in dict2.items():
  print(i,':',j)

text-1
identify entities (person, place, thing, location, work of art)
person : [Office, Greg Daniels, Simpsons, Daniels, Daniels, Howard Klein, Ben Silverman, Ricky Gervais, Stephen Merchant, Padmarajan, Bharathan, Padmarajan, Padmarajan]
place : [Scranton, Pennsylvania]
thing : []
location : [Scranton, Pennsylvania]
work of art : [Saturday Night Live, King of the Hill, Moonnam Pakkam, Innale, Njan Gandharvan]
Categorize them into ‘living entity’, ‘person’ or ‘object’.
living entity : []
person : [Office, Greg Daniels, Simpsons, Daniels, Daniels, Howard Klein, Ben Silverman, Ricky Gervais, Stephen Merchant, Padmarajan, Bharathan, Padmarajan, Padmarajan]
object : [Saturday Night Live, King of the Hill, Moonnam Pakkam, Innale, Njan Gandharvan, Scranton, Pennsylvania]
